In [1]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.2.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /Users/boramert/opt/anaconda3/lib/python3.9/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: pyod, scikit-learn-intelex


In [2]:
import sys
sys.path.append('/Users/boramert/opt/anaconda3/lib/python3.9/site-packages')

In [3]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import regularizers, Sequential

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

import matplotlib.pyplot as plt
from matplotlib import gridspec

from scipy import stats

import seaborn as sns

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

/Users/boramert/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def labelencoder(ds):
    encoded_ds=ds.copy()
    le = preprocessing.LabelEncoder()
    for col in encoded_ds.select_dtypes(include=['object']).columns:
        encoded_ds[col]=le.fit_transform(encoded_ds[col])
    
    return encoded_ds

def encoding_for_anomalous(ds):
    encoded_ds=ds.copy()
    for i in range(len(ds)):
        if encoded_ds.iloc[i]=='BENIGN':
            encoded_ds.iloc[i]= 0
        else:
            encoded_ds.iloc[i]= 1
            
    return encoded_ds  

def scaling(ds):
    scaler=preprocessing.StandardScaler()
    scaled_ds=scaler.fit_transform(ds)
    return scaled_ds

def drop_infs(ds):
    ds.replace([np.inf, -np.inf], np.nan, inplace=True)
    ds.dropna(how='any', inplace=True)
    return ds

In [5]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [6]:
dataset=pd.read_csv("test_dataset.csv")

In [7]:
dataset=drop_infs(dataset)
y=dataset[' Label']
x=dataset.drop(' Label', axis=1)
x_scaled=scaling(x)

In [8]:
y.value_counts()

0    1741839
1     556556
Name:  Label, dtype: int64

In [9]:
x_normal = x_scaled[y == 0]
x_anomaly = x_scaled[y == 1]

In [10]:
x_normal_train = np.asarray(x_normal).astype(np.float32)

x_normal_train = tf.cast(x_normal_train, tf.float32)

2022-12-12 22:38:49.180642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
from tensorflow.keras.optimizers import Adam
autoencoder = Sequential()
autoencoder.add(Dense(32,  activation='relu', input_shape=(78,)))
autoencoder.add(Dense(16,  activation='relu'))
autoencoder.add(Dense(8,    activation='linear', name="Compressed"))
autoencoder.add(Dense(16,  activation='relu'))
autoencoder.add(Dense(32,  activation='relu'))
autoencoder.add(Dense(78,  activation='sigmoid'))
autoencoder.compile(loss='mean_squared_error', optimizer = 'adam')

In [12]:
history = autoencoder.fit(x_normal_train, 
                          x_normal_train, 
                          batch_size=64,     
                          epochs=10, 
                          verbose=1, 
                          validation_split = 0.20)

Epoch 1/10
21773/21773 [==============================] - 19s 825us/step - loss: 0.7424 - val_loss: 0.3437
Epoch 2/10
21773/21773 [==============================] - 20s 927us/step - loss: 0.7361 - val_loss: 0.3433
Epoch 3/10
21773/21773 [==============================] - 17s 780us/step - loss: 0.7357 - val_loss: 0.3436
Epoch 4/10
21773/21773 [==============================] - 17s 776us/step - loss: 0.7355 - val_loss: 0.3433
Epoch 5/10
21773/21773 [==============================] - 17s 778us/step - loss: 0.7353 - val_loss: 0.3433
Epoch 6/10
21773/21773 [==============================] - 17s 774us/step - loss: 0.7352 - val_loss: 0.3432
Epoch 7/10
21773/21773 [==============================] - 17s 777us/step - loss: 0.7351 - val_loss: 0.3430
Epoch 8/10
21773/21773 [==============================] - 18s 832us/step - loss: 0.7350 - val_loss: 0.3429
Epoch 9/10
21773/21773 [==============================] - 19s 857us/step - loss: 0.7349 - val_loss: 0.3429
Epoch 10/10
21773/21773 [============

In [14]:
encoder = Model(autoencoder.input, autoencoder.get_layer('Compressed').output)

In [15]:
encoded_normal = encoder.predict(x_normal)
encoded_anomaly = encoder.predict(x_anomaly)

17393/17393 [==============================] - 6s 331us/step


In [17]:
encoded_x = np.append(encoded_normal, encoded_anomaly, axis = 0)
normal_y = np.zeros(encoded_normal.shape[0])
anomaly_y = np.ones(encoded_anomaly.shape[0])
encoded_y = np.append(normal_y, anomaly_y)

In [18]:
x_train, x_val, y_train, y_val = train_test_split(encoded_x, encoded_y, test_size=0.25)

In [20]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.0 MB/s eta 0:00:0000:0100:01


In [21]:
from sklearn.linear_model import RidgeCV, ElasticNet, LassoCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
model_xgb = xgb.XGBClassifier(n_estimators=380, max_depth=2, learning_rate=0.2)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model_xgb, x_train, y_train, scoring='accuracy', cv=cv, error_score='raise')

print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
model_xgb.fit(x_train, y_train)
xgb_preds = model_xgb.predict(x_val)

In [ ]:
model_xgb.score(x_val, y_val)

In [ ]:
print ("")
print ("Classification Report: ")
print (classification_report(YVal, xgb_preds))

print ("")
print ("Accuracy Score: ", accuracy_score(y_val, xgb_preds))

In [ ]:
from sklearn.metrics import confusion_matrix 

In [ ]:
LABELS = ['Normal', 'Fraud'] 
conf_matrix = confusion_matrix(y_val, xgb_preds) 
plt.figure(figsize =(12, 12)) 
sns.heatmap(conf_matrix, xticklabels = LABELS,  
            yticklabels = LABELS, annot = True, fmt ="d"); 
plt.title("Confusion matrix") 
plt.ylabel('True class')
plt.xlabel('Predicted class') 
plt.show()